In [2]:
from pprint import pprint
import re
import requests
from pymongo import MongoClient
from bson import ObjectId
import re


regex = re.compile('https?://t\.co/[A-z0-9]*')

conn = MongoClient('laisky.com', port=27016)
db=conn.twitter

## Format Url

In [ ]:
url_regex = re.compile(
    r"(?xi)"
    "\b"
    "("                                     # Capture 1: entire matched URL
    "(?:"
    "https?://"                             # http or https protocol
    "|"                                     # or
    "www\d{0,3}[.]"                         # "www.", "www1.", "www2." … "www999."
    "|"                                     # or
    "[a-z0-9.\-]+[.][a-z]{2,4}/"            # looks like domain name followed by a slash
    ")"
    "(?:"                                   # One or more:
    "[^\s()<>]+"                            # Run of non-space, non-()<>
    "|"                                     # or
    "\(([^\s()<>]+|(\([^\s()<>]+\)))*\)"    # balanced parens, up to 2 levels
    ")+"
    "(?:"                                   # End with:
    "\(([^\s()<>]+|(\([^\s()<>]+\)))*\)"    # balanced parens, up to 2 levels
    "|"                                     # or
    "[^\s`!()\[\]{};:'\".,<>?«»\“\”‘’]"     # not a space or one of these punct chars
    ")"
    ")"
)

def find_urls(text):
    return url_regex.findall(text)
    

count = 0
for docu in db.tweets.find(timeout=False):
    t = docu['text']
    
    if 'http' not in t:
        continue
    
    if count % 50 == 0:
        print('finished {}'.format(count))
        
    t = docu['text']
    
    for i, surl in enumerate(find_urls(t)):
        ul = len(surl)
        ui = t.index(surl)
        if ul + ui != len(t):
            if t[ul + ui] != ' ':
                t = t[:ui + ul] + ' ' + t[ui + ul:]

        if ui != 0:
            if t[ui - 1] != ' ':
                t = t[:ui] + ' ' + t[ui:]

#     pprint(docu)
#     break
                
    docu['text'] = t
    _id = docu.pop('_id')
    db.tweets.update({'_id': _id}, {'$set': docu})
    count += 1

## 导出的历史推文

docu 中 `expanded_urls` 字段

In [11]:
count = 0
for docu in db.tweets.find():
    if 'expanded_urls' not in docu['text'] or 't.co' not in docu['text']:
        continue
        
    if count % 50 == 0:
        print('finished {}'.format(count))
        
    t = docu['text']
    if not docu['expanded_urls']:
        pprint(docu)
        
    eurls = docu['expanded_urls'].split(',')
    surls = regex.findall(t)
    if len(eurls) != len(surls):
        _u = set()
        eurls = [u for u in eurls if u not in _u and not _u.add(u)]
        if len(eurls) != len(surls):
            continue
            print('ERROR! for docu {}'.format(docu))
            break
            
    for i, surl in enumerate(surls):
        ul = len(surl)
        ui = t.index(surl)
        if ul + ui != len(t):
            if t[ul + ui] != ' ':
                t = t[:ui + ul] + ' ' + t[ui + ul:]
                
        if ui != 0:
            if t[ui - 1] != ' ':
                t = t[:ui] + ' ' + t[ui:]
        
        t = t.replace(surl, eurls[i])
        
    docu['text'] = t
#     print(t)
    _id = docu.pop('_id')
    db.tweets.update({'_id': _id}, {'$set': docu})
    count += 1
#     if count > 50:
#         break
        


## API 获得的新推文

In [8]:
count = 0
for docu in db.tweets.find({'entities': {'$exists': 1}}):
    t = docu['text']
    if 't.co' not in t:
        continue
        
    if count % 50 == 0:
        print('finished {}'.format(count))
        
    t = docu['text']
    surls = []
    eurls = []
    
    # parse entities media
    if 'media' in docu['entities']:
        for media in docu['entities']['media']:
            surl = media['url']
            eurl = media['media_url']
            t = t.replace(surl, eurl)
        
    # parse entities urls
    if 'urls' in docu['entities']:
        for d in docu['entities']['urls']:
            surl = d['url']
            eurl = d['expanded_url']
            t = t.replace(surl, eurl)
            
    docu['text'] = t
    _id = docu.pop('_id')
    db.tweets.update({'_id': _id}, {'$set': docu})
    count += 1
        



finished 0
{'_id': ObjectId('5491176e98dc5b91bc0583a2'),
 'contributors': None,
 'coordinates': None,
 'created_at': datetime.datetime(2014, 12, 17, 5, 40, 56),
 'entities': {'hashtags': [],
              'media': [{'display_url': 'pic.twitter.com/sbouqsUjcW',
                         'expanded_url': 'http://twitter.com/examisboring/status/545021141564133376/photo/1',
                         'id': 545021140242952192,
                         'id_str': '545021140242952192',
                         'indices': [60, 82],
                         'media_url': 'http://pbs.twimg.com/media/B5BN1uNCcAAKRFi.jpg',
                         'media_url_https': 'https://pbs.twimg.com/media/B5BN1uNCcAAKRFi.jpg',
                         'sizes': {'large': {'h': 750,
                                             'resize': 'fit',
                                             'w': 422},
                                   'medium': {'h': 750,
                                              'resize': 'fit',


## 随手调试

In [5]:
docu = db.tweets.find_one({'_id': ObjectId('54a74d222374fca232bfa818')})
# expanded_urls = docu['expanded_urls']
text = docu['text']
# pprint(docu['entities'])
pprint(docu)


{'_id': ObjectId('54a74d222374fca232bfa818'),
 'contributors': None,
 'coordinates': None,
 'created_at': datetime.datetime(2015, 1, 3, 1, 23, 40),
 'entities': {'hashtags': [],
              'media': [{'display_url': 'pic.twitter.com/uTWrfuDEz7',
                         'expanded_url': 'http://twitter.com/NASA/status/551185158578397184/photo/1',
                         'id': 551185158108631040,
                         'id_str': '551185158108631040',
                         'indices': [139, 140],
                         'media_url': 'http://pbs.twimg.com/media/B6Yz-yKIUAABy7x.jpg',
                         'media_url_https': 'https://pbs.twimg.com/media/B6Yz-yKIUAABy7x.jpg',
                         'sizes': {'large': {'h': 576,
                                             'resize': 'fit',
                                             'w': 1024},
                                   'medium': {'h': 337,
                                              'resize': 'fit',
                  